In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])  # ImageNet标准化
])
transform_train = transforms.Compose([
    transforms.Resize((40, 40)),  # 先放大
    transforms.RandomCrop(32),     # 再随机裁剪
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])  # ImageNet标准化
])
batch_size=4
trainset=torchvision.datasets.CIFAR10(root=r'./',train=True,download=True,transform=transform_train)
testset=torchvision.datasets.CIFAR10(root=r'./',train=False,download=True,transform=transform)
trainloader=torch.utils.data.DataLoader(dataset=trainset,batch_size=batch_size,shuffle=True,num_workers=2)
testloader=torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False,num_workers=2)
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

100%|██████████| 170M/170M [00:03<00:00, 53.6MB/s]


In [2]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        # 主路径
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # 捷径连接
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # 残差连接
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_channels = 64
        
        # 初始卷积层（适应CIFAR-10的32x32尺寸）
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # 残差块层
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)
        
        # 全局平均池化和全连接层
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
    
    def _make_layer(self, block, out_channels, num_blocks, stride):
        # 创建包含多个残差块的层
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels, stride=1))
        
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = nn.ReLU()(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [3]:
import matplotlib.pyplot as plt
model=ResNet18().to(device)
LR=0.0001
criterion=nn.CrossEntropyLoss().cuda()
optimizer=optim.Adam(model.parameters(),LR,weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 
    T_0=10,     # 第一次重启的周期
    T_mult=2,   # 每次重启后周期倍增
    eta_min=1e-6  # 最小学习率
)
max_grad_norm = 1.0  # 梯度裁剪阈值
# 添加图像描述变量
image_captions = []

# 创建存储训练指标的列表
train_losses = []
val_losses = []  # 新增验证损失列表
val_accuracies = []
learning_rates = []
epochs=30
for epoch in range(epochs):
    running_loss=0.0
    total=0
    correct=0
    model.train()
    # 添加当前epoch的图像描述
    epoch_caption = f"Epoch {epoch+1}/{epochs}: Training in progress..."
    image_captions.append(epoch_caption)
    for i,data in enumerate(trainloader,0):
        inputs,labels=data
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        running_loss+=loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if i % 2000 == 1999:  # 每2000个batch打印一次
            train_acc = 100 * correct / total
            batch_caption = (f'Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(trainloader)}], '
                            f'Loss: {running_loss/2000:.4f}, Acc: {train_acc:.2f}%')
            print(batch_caption)
            image_captions.append(batch_caption)
            running_loss = 0.0
            total = 0
            correct = 0
            
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        learning_rates.append(current_lr)

     # 验证过程
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_acc = 100 * val_correct / val_total
    avg_val_loss = val_loss / len(testloader)
    val_losses.append(avg_val_loss)  # 记录验证损失
    val_accuracies.append(val_acc)
    
    epoch_summary = (f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}, '
                    f'Accuracy: {val_acc:.2f}%, LR: {current_lr:.6f}')
    print(epoch_summary)
    image_captions.append(epoch_summary)
    train_losses.append(val_loss)
    
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%, LR: {current_lr:.6f}')


print('Finished Training')

Epoch [1/30], Batch [2000/12500], Loss: 2.0560, Acc: 24.48%
Epoch [1/30], Batch [4000/12500], Loss: 1.8791, Acc: 31.51%
Epoch [1/30], Batch [6000/12500], Loss: 1.7467, Acc: 36.85%
Epoch [1/30], Batch [8000/12500], Loss: 1.6657, Acc: 40.04%
Epoch [1/30], Batch [10000/12500], Loss: 1.4743, Acc: 48.33%
Epoch [1/30], Batch [12000/12500], Loss: 1.5766, Acc: 44.30%
Epoch 1, Validation Loss: 1.3054, Accuracy: 53.67%, LR: 0.000088
Epoch 1, Validation Loss: 3263.5979, Accuracy: 53.67%, LR: 0.000088
Epoch [2/30], Batch [2000/12500], Loss: 1.4960, Acc: 46.17%
Epoch [2/30], Batch [4000/12500], Loss: 1.3794, Acc: 51.14%
Epoch [2/30], Batch [6000/12500], Loss: 1.2730, Acc: 55.06%
Epoch [2/30], Batch [8000/12500], Loss: 1.2264, Acc: 56.66%
Epoch [2/30], Batch [10000/12500], Loss: 1.4201, Acc: 50.10%
Epoch [2/30], Batch [12000/12500], Loss: 1.3933, Acc: 50.71%
Epoch 2, Validation Loss: 1.3026, Accuracy: 56.52%, LR: 0.000089
Epoch 2, Validation Loss: 3256.4944, Accuracy: 56.52%, LR: 0.000089
Epoch [3/3

In [4]:
# 保存最终模型
torch.save(model.state_dict(), 'resnet_model.pth')
final_caption = "Training completed. Model saved as final_model.pth"
print(final_caption)
image_captions.append(final_caption)

# 绘制测试集准确率变化折线图
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs+1), val_accuracies, 'b-o', linewidth=2)
plt.title('Test Accuracy Over Epochs', fontsize=14)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(1, epochs+1))
plt.ylim(0, 100)  # 确保y轴从0到100%

# 标记最高准确率
max_acc = max(val_accuracies)
max_epoch = val_accuracies.index(max_acc) + 1
plt.annotate(f'Max: {max_acc:.2f}%', 
             xy=(max_epoch, max_acc),
             xytext=(max_epoch+1, max_acc-5),
             arrowprops=dict(facecolor='red', shrink=0.05),
             fontsize=12)

# 保存图表
plt.savefig('accuracy_plot.png', dpi=300, bbox_inches='tight')
plt.close()

# 添加图表描述
plot_caption = ("Accuracy Plot: Shows the model's performance improvement on the test set over training epochs. "
               f"Highest accuracy of {max_acc:.2f}% achieved at epoch {max_epoch}.")
image_captions.append(plot_caption)

# 保存所有图像描述到文件
with open('training_report.txt', 'w') as f:
    for caption in image_captions:
        f.write(caption + '\n')

print("Training report and accuracy plot saved successfully.")

Training completed. Model saved as final_model.pth
Training report and accuracy plot saved successfully.
